In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import altair as alt

In [3]:
from ecostyles import EcoStyles
# Create styles instance
styles = EcoStyles()
# Register and enable a theme
styles.register_and_enable_theme(theme_name="article")  # or "article"

In [16]:
uk = pd.read_csv('/Users/sambickel-barlow/RADataHub/RADataHub/ChartOfTheDay/inflation/series-260825.csv', skiprows=501).rename(columns={'2014 DEC': 'Date',"0.5": "Value"})
ez = pd.read_csv('/Users/sambickel-barlow/RADataHub/RADataHub/ChartOfTheDay/inflation/ECB Data Portal_20250826162819.csv', skiprows=216).rename(columns={'2014-12-31': 'Date',"2014Dec": "drop","-0.2":"Value"})
us = pd.read_csv('/Users/sambickel-barlow/RADataHub/RADataHub/ChartOfTheDay/inflation/CPIAUCSL.csv').rename(columns={'observation_date': 'Date',"CPIAUCSL":"raw value"})    

In [17]:
uk['Date'] = pd.to_datetime(uk['Date'], format='%Y %b')

In [18]:
ez = ez[['Date','Value']]
ez['Date'] = pd.to_datetime(ez['Date']).dt.to_period('M').dt.start_time

In [19]:
us['Date'] = pd.to_datetime(us['Date'], format='%Y-%m-%d')
us['Value'] = 0
for i in range(0,len(us)):
    us['Value'].iloc[i] = ((us['raw value'].iloc[i] - us['raw value'].iloc[i - 12]) / us['raw value'].iloc[i - 12]) * 100
us = us[['Date','Value']][12:]
us['Value'] = us['Value'].round(1)

In [20]:
infl = uk.merge(ez, how='outer', on='Date').merge(us, how='outer', on='Date').rename(columns={'Value_x': 'UK', 'Value_y': 'EZ', 'Value': 'US'})
infl = infl.melt('Date', var_name='Country', value_name='Value')
infl['Value'] = infl['Value'] / 100
infl = infl[infl['Date'] >= '2024-08-01']

In [21]:
# Custom sort so drawing order is US, EU, UK (UK will be drawn last/on top)
country_order = {'US': 0, 'EZ': 1, 'UK': 2}
infl['sort_order'] = infl['Country'].map(country_order)
infl = infl.sort_values(['sort_order', 'Date']).drop('sort_order', axis=1)

In [23]:
# 2. Find last year for each Type
last_points = infl.loc[infl.groupby('Country')['Date'].idxmax()]

line = alt.Chart(infl).mark_line().encode(
    x=alt.X('Date', axis=alt.Axis(format='%b%y', title=None, labelAngle=0, tickCount={'interval': 'month', 'step': 1})),
    y=alt.Y('Value', title='Inflation Rate', axis=alt.Axis(format=',.1%', titleX=-87, titleY=-10, titleFontSize=13)),
    color='Country',
    tooltip=[
        alt.Tooltip('Country:N', title='Economy'),
        alt.Tooltip('Date:T', format='%B %Y', title='Date'),
        alt.Tooltip('Value:Q', format='.1%', title='Inflation Rate')
    ]
)

# 4. Add text labels at the end
text = alt.Chart(last_points).mark_text(
    align='left',
    dx=5,
    fontWeight='bold',
    fontSize=15
).encode(
    x=alt.X('Date'),
    y=alt.Y('Value'),
    text='Country',
    color=alt.Color('Country', legend=None)
)

footnote = alt.Chart(pd.DataFrame({'text': [
    "Sources: ONS, ECB, FRED"
]})).mark_text(
    align='left',
    baseline='top',
    fontSize=14,
    fontStyle='italic',
    dy=175,
    x=-15
).encode(
    text='text'
)

# 5. Combine and display
chart = (line + text + footnote).properties(title= 'Headline Inflation Rate - UK, US, and Eurozone', width=600).configure_title(
    fontSize=20  # Increase title font size here
)
chart

alt.LayerChart(...)

In [24]:
# Save to png
chart.save('Infl_comp.png', scale_factor=2)
chart.save('Infl_comp.json', scale_factor=2)